In [83]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import math
import datetime
from sklearn.model_selection import train_test_split 

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
#pd.set_option('max_colwidth', 700)

# Import data

In [69]:
data = pd.read_csv('../data/Gelderman_SOD_cohort/data_w_embs')
print(len(data))
display(data.head())
display()

42080


,new_id,img_head,correct_img_date,date_placed_ARF,PMI_days,age_at_death,sex,cadaver_weight_lbs,est_stature_in,true_SOD_G_head,BP_of_true_SOD_G,donor_date,img_torso,true_SOD_G_torso,img_limbs,true_SOD_G_limbs,index,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63
0,004,00400124.12.JPG,2018-01-24,2018-01-09,15.0,67.0,Female,171.0,64.0,3.0,head,00400124,00400124.07.JPG,3.0,00400124.10.JPG,3.0,0,0.724824,1.149898,0.689984,1.239605,2.823720,0.768867,4.392087,2.289970,0.854456,1.994645,0.878973,1.475379,0.0,1.034006,0.745903,1.022248,2.885059,1.058317,1.267659,2.312767,0.000000,0.000000,1.503174,1.252649,1.465867,0.029327,2.485211,1.145958,0.000000,0.623376,0.753480,1.694158,1.114506,1.730707,0.0,0.560717,0.990937,3.373392,1.368024,0.0,1.433687,0.468360,0.0,1.076626,0.0,0.400011,3.335856,0.639417,0.788177,0.0,0.354045,3.52474,2.901803,1.90379,3.055927,0.356006,0.0,0.212531,1.343125,2.152382,1.062701,1.635706,1.340923,0.0
1,004,00400124.12.JPG,2018-01-24,2018-01-09,15.0,67.0,Female,171.0,64.0,3.0,head,00400124,00400124.07.JPG,3.0,00400124.11.JPG,3.0,1,0.716963,1.149898,0.955302,1.319903,2.705828,0.768867,4.392087,2.307821,0.854456,1.504295,0.878973,1.201398,0.0,0.896066,0.745903,0.907396,3.021500,0.961600,1.267659,2.386908,0.000000,0.000000,1.317928,1.525690,1.118749,0.061520,2.485211,0.965051,0.000000,0.684890,0.753480,1.694158,0.892880,1.786678,0.0,0.560717,0.890066,3.153000,1.368024,0.0,1.433687,0.393529,0.0,0.922296,0.0,0.400011,3.096745,0.562556,0.788177,0.0,0.354045,3.52474,2.901803,1.90379,3.200524,0.356006,0.0,0.178267,1.343125,2.153842,0.962503,1.635706,1.340923,0.0
2,004,00400124.12.JPG,2018-01-24,2018-01-09,15.0,67.0,Female,171.0,64.0,3.0,head,00400124,00400124.07.JPG,3.0,00400124.18.JPG,4.0,2,0.000000,1.149898,0.000000,1.239605,3.062716,0.768867,4.392087,1.867076,0.854456,2.965985,0.878973,1.265600,0.0,2.985819,0.745903,1.480414,2.043196,0.959247,1.267659,2.312767,0.000000,0.259414,2.555381,1.050051,2.345941,0.029327,4.311337,2.085147,0.206266,0.623376,1.085992,1.694158,2.532620,1.612098,0.0,0.560717,0.210517,3.469567,1.368024,0.0,1.433687,0.366301,0.0,3.380263,0.0,0.697988,3.412217,2.328625,0.788177,0.0,0.354045,3.52474,2.901803,1.90379,2.886595,0.356006,0.0,0.000000,1.343125,2.997689,1.215456,1.635706,1.340923,0.0
3,004,00400124.12.JPG,2018-01-24,2018-01-09,15.0,67.0,Female,171.0,64.0,3.0,head,00400124,00400124.07.JPG,3.0,00400124.19.JPG,4.0,3,0.000000,1.149898,0.000000,1.239605,3.025267,0.768867,4.392087,1.867076,0.854456,2.883476,0.878973,1.350307,0.0,2.912200,0.745903,1.394427,2.043196,0.959247,1.267659,2.312767,0.000000,0.410184,2.610728,1.050051,2.318987,0.029327,4.337298,1.892848,0.153347,0.623376,1.375643,1.694158,2.450993,1.612098,0.0,0.560717,0.210517,3.338618,1.368024,0.0,1.433687,0.409887,0.0,3.286015,0.0,0.548375,3.403030,2.268347,0.788177,0.0,0.354045,3.52474,2.901803,1.90379,2.886595,0.356006,0.0,0.000000,1.343125,2.973740,1.137876,1.635706,1.340923,0.0
4,004,00400124.12.JPG,2018-01-24,2018-01-09,15.0,67.0,Female,171.0,64.0,3.0,head,00400124,00400124.07.JPG,3.0,00400124.36.JPG,3.0,4,0.867568,1.149898,0.503202,1.239605,2.375831,0.875600,4.392087,2.232578,0.854456,1.602763,0.878973,1.289241,0.0,0.851745,0.745903,0.785885,2.401414,0.959247,1.267659,2.312767,0.253486,0.000000,1.102267,1.580117,1.318771,0.075269,2.485211,0.914396,0.000000,0.623376,0.816278,1.694158,0.308897,2.233836,0.0,0.560717,0.655829,3.430094,1.368024,0.0,1.433687,0.757137,0.0,0.859736,0.0,0.400011,2.903947,0.549988,0.788177,0.0,0.354045,3.52474,2.901803,1.90379,3.249255,0.356006,0.0,0.125022,1.343125,2.538878,1.060257,1.635706,1.340923,0.0


### Downsize data 

In [70]:
data = data.groupby('donor_date').sample(n=6, random_state=1, replace=True).copy()
len(data)

1044

# Process data

### Add season_of_recovery feature

In [71]:
# create season_of_recovery
data['month'] = data['donor_date'].str[-4:-2]
print(data.month.unique())

winter = ['12', '01', '02']
spring = ['03', '04', '05']
summer = ['06', '07', '08']
fall = ['09', '10', '11']


conditions = [
    (data['month'].isin(winter)),
    (data['month'].isin(spring)),
    (data['month'].isin(summer)),
    (data['month'].isin(fall))
    ]

values = ['winter', 'spring', 'summer', 'fall']

data['season_of_recovery'] = np.select(conditions, values)
display(data.sample(10))

['01' '02' '05' '08' '12' '04' '03' '07' '06' '09' '11' '10']


,new_id,img_head,correct_img_date,date_placed_ARF,PMI_days,age_at_death,sex,cadaver_weight_lbs,est_stature_in,true_SOD_G_head,BP_of_true_SOD_G,donor_date,img_torso,true_SOD_G_torso,img_limbs,true_SOD_G_limbs,index,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,month,season_of_recovery
42062,ff9,ff900306.25.JPG,2017-03-06,2017-02-09,25.0,68.0,Male,89.0,66.0,3.0,head,ff900306,ff900306.07.JPG,3.0,ff900306.31.JPG,2.0,42062,2.283834,0.998722,1.643833,2.305868,2.417286,1.808994,2.439875,3.322491,1.247074,1.357219,0.000000,1.742782,0.000000,0.910117,1.208060,0.165549,2.991841,1.959138,0.958266,4.762515,0.000000,0.130988,0.481658,2.093827,1.645186,0.000000,3.028074,2.509511,0.151755,0.000000,0.378962,2.323677,1.608356,1.635415,0.235926,0.786468,3.870776,2.547090,3.725987,0.0,2.580447,0.666033,0.029553,0.000000,0.028751,0.218113,2.933687,0.508833,0.636314,0.083842,1.732091,3.465558,2.048682,1.022560,3.992064,0.733737,0.050193,1.571614,1.199929,2.498932,0.000000,1.540047,0.702007,0.000000,03,spring
36181,c2d,c2d01124.23.JPG,2017-11-24,2017-11-22,2.0,85.0,Male,116.0,68.0,2.0,head,c2d01124,c2d01124.28.JPG,2.0,c2d01124.11.JPG,4.0,36181,0.000000,1.302030,1.489130,1.038932,1.759462,0.088865,1.316719,3.392354,0.000000,1.336387,0.733071,1.774727,0.000000,2.698462,0.000000,0.962082,2.587566,0.000000,0.000000,1.373462,0.104349,1.195051,1.232085,0.853243,0.000000,0.743151,2.140776,1.889368,0.000000,2.250409,0.867723,0.824718,2.525800,0.000000,0.000000,0.000000,2.233826,2.309794,1.219499,0.0,2.092182,0.000000,0.245736,2.609562,0.000000,0.543356,2.035814,1.715809,2.142094,1.481531,0.000000,2.281995,2.596302,1.425667,0.806617,0.998366,0.323943,0.172181,0.998067,0.726235,0.476080,0.247972,0.000000,1.016319,11,fall
16367,47d,47d00912.29.JPG,2012-09-12,2012-07-16,58.0,75.0,Male,NaN,71.0,5.0,head,47d00912,47d00912.05.JPG,4.0,47d00912.30.JPG,5.0,16367,0.144069,1.576744,0.000000,1.214750,1.020508,2.135123,0.756562,0.844786,0.983961,0.932446,0.000000,1.465278,1.481917,4.224247,0.000000,0.713904,0.490924,1.251076,1.137822,1.484892,0.000000,0.206959,1.435878,1.621521,2.485882,0.000000,2.328421,1.494089,2.139944,3.043307,0.596339,1.280148,1.225349,2.499415,0.268595,0.890796,0.000000,0.226694,0.247213,0.0,1.223948,0.845250,0.124658,2.464853,1.016930,1.242044,0.196038,3.164133,0.000000,1.422284,2.435866,0.000000,0.179699,0.950941,0.300837,0.699797,0.927160,0.218880,2.434800,2.621180,1.423365,1.467067,0.000000,0.909440,09,fall
32098,a5a,a5a00902.47.JPG,2013-09-02,2013-07-24,40.0,31.0,Male,104.0,70.0,6.0,head,a5a00902,a5a00902.12.JPG,5.0,a5a00902.45.JPG,5.0,32098,0.242333,0.254431,0.000000,0.623704,2.870371,4.500565,1.249199,0.097194,0.308459,2.843352,1.092096,4.859176,3.549711,4.863742,0.000000,0.635876,0.002315,0.228252,0.468740,1.946965,0.114248,1.199736,0.633547,0.000000,4.122959,0.000000,2.931563,1.179420,2.729310,0.297595,2.998952,0.478040,0.619968,1.471822,0.000000,0.899745,0.267818,0.560723,0.991800,0.0,0.711364,1.520754,0.000000,3.605565,2.238210,1.176109,0.000000,3.207461,0.000000,0.000000,0.941138,0.000000,0.016416,1.591398,0.000000,1.805513,1.101464,1.530709,0.958806,2.264672,1.444749,4.042343,0.000000,1.987164,09,fall
13127,215,21500528.05.JPG,2014-05-28,2014-03-07,82.0,44.0,Female,76.0,63.0,3.0,head,21500528,21500528.07.JPG,3.0,21500528.09.JPG,3.0,13127,0.816996,0.659339,0.000000,1.102105,1.619142,1.779006,2.293690,2.190101,1.060521,1.871482,0.051363,1.421847,0.657715,1.560983,0.951626,1.192477,1.759584,0.717426,0.680381,1.696336,0.928818,0.000000,1.323657,1.160118,2.438760,0.000000,2.546303,0.575952,0.269071,0.000000,0.332266,2.173554,0.510316,1.242633,0.090357,0.636097,0.816665,3.298100,0.879989,0.0,0.629831,1.278286,0.000000,0.875723,0.483155,0.651883,3.007781,1.062191,0.000000,0.000000,1.348302,1.675368,2.118156,1.143464,1.739170,0.020582,0.561844,0.314217,0.975778,3.015718,0.634475,1.644541,1.253185,0.000000,05,

In [72]:
# season_of_recovery to numeric columns
dummies = pd.get_dummies(data.season_of_recovery)

# concatenate the dummies to original dataframe
data = pd.concat([data, dummies], axis='columns').copy()
data.head()

# drop the values
data.drop(['winter'], axis='columns', inplace=True)
data.sample(4)

,new_id,img_head,correct_img_date,date_placed_ARF,PMI_days,age_at_death,sex,cadaver_weight_lbs,est_stature_in,true_SOD_G_head,BP_of_true_SOD_G,donor_date,img_torso,true_SOD_G_torso,img_limbs,true_SOD_G_limbs,index,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,month,season_of_recovery,fall,spring,summer
41451,f2d,f2d00714.08.JPG,2016-07-14,2016-03-07,129.0,77.0,Female,186.0,63.0,6.0,head,f2d00714,f2d00714.21.JPG,5.0,f2d00714.12.JPG,5.0,41451,0.568604,0.213792,0.087450,0.00000,1.163980,3.785584,0.000000,0.310295,0.000000,1.684781,1.106335,1.935194,1.819091,2.735841,0.000000,0.000000,0.000000,0.640774,0.318806,0.791161,0.605853,0.124671,0.000000,0.000000,2.655952,0.019715,0.780888,1.063367,1.400839,0.578804,0.631787,0.387114,1.074375,1.414368,0.305325,0.885501,0.047125,0.353562,1.341402,0.0,1.533840,0.559912,0.000000,1.663046,1.808245,1.210946,0.194360,1.845080,0.000000,0.313007,1.109522,0.000000,0.095185,0.768411,0.335263,1.580751,0.392385,0.920288,1.177848,2.176009,0.880528,1.980451,0.00000,1.275372,07,summer,0,0,1
29004,9f7,9f700715.06.JPG,2015-07-15,2015-06-05,40.0,85.0,Male,216.0,72.0,4.0,head,9f700715,9f700715.16.JPG,4.0,9f700715.28.JPG,4.0,29004,0.000000,1.409608,0.488870,1.68439,2.467956,1.296870,1.567743,2.399577,1.342265,1.321561,0.000000,0.700198,0.038651,3.348260,0.747503,1.542371,2.394469,1.439572,0.086171,1.301141,0.000000,0.141407,2.161525,1.493263,1.108157,0.000000,1.304214,0.561280,0.000000,0.882901,0.338921,1.217533,1.346064,1.652442,0.031380,0.328363,0.411583,1.294389,0.017843,0.0,0.566489,1.039539,0.000000,2.087214,0.472547,0.770762,2.652798,1.465703,0.055592,0.026565,0.974076,0.296047,1.074943,0.848575,0.818504,0.000000,0.669166,0.000000,2.226541,1.483490,1.179208,2.082940,1.00242,0.474264,07,summer,0,0,1
13776,28e,28e00422.25.JPG,2015-04-22,2015-04-20,2.0,68.0,Male,147.0,64.0,2.0,head,28e00422,28e00422.20.JPG,2.0,28e00422.37.JPG,2.0,13776,2.857548,0.310792,1.307667,1.84378,0.725969,0.031380,0.853572,2.264410,0.634552,2.314159,1.458540,3.934143,0.000000,0.000000,0.696702,0.731887,0.870724,0.840818,0.000000,4.059414,0.448696,2.308694,1.520038,0.310447,1.432609,0.490849,2.912455,1.581054,0.184689,1.566196,0.257253,1.378494,1.402788,2.273100,1.492272,0.696445,6.335750,0.341203,4.090106,0.0,2.529892,0.000000,1.406247,0.000000,0.000000,0.063353,2.641536,0.116692,3.132831,0.936778,1.401796,3.124651,2.545091,1.161192,3.992256,1.511811,0.000000,1.243374,1.860256,0.046314,0.303618,0.428036,0.00000,1.240419,04,spring,0,1,0
17528,598,59801013.13.JPG,2014-10-13,2014-07-23,82.0,69.0,Male,194.0,70.0,6.0,head,59801013,59801013.18.JPG,5.0,59801013.46.JPG,4.0,17528,0.720193,0.000000,0.056486,0.09126,2.219064,3.606930,0.390947,0.577023,0.000000,2.700209,1.935341,3.247416,1.687736,2.502534,0.043156,1.242531,0.714827,0.000000,0.110830,1.011971,0.000000,0.397225,1.112533,0.000000,1.980474,0.612568,2.339687,0.324359,0.302147,0.991875,1.298625,1.119259,2.321685,1.198096,0.000000,0.000000,0.375391,1.940932,0.101034,0.0,2.088284,1.098751,0.040474,3.003840,2.388889,0.705487,1.534198,2.271929,0.000000,0.497999,1.481666,1.019141,0.000000,2.092384,0.000000,1.976904,1.348852,1.398620,0.878821,2.338565,2.511160,1.663571,0.00000,1.107624,10,fall,1,0,0


### Process sex feature

In [73]:
data.sex.unique()

array(['Female', 'Male'], dtype=object)

In [74]:
# sex column to numeric
data['sex_male'] = np.where(data['sex'] == "Male", 1, 0).copy()
data.drop(['sex'], axis=1, inplace=True)
data.head()

,new_id,img_head,correct_img_date,date_placed_ARF,PMI_days,age_at_death,cadaver_weight_lbs,est_stature_in,true_SOD_G_head,BP_of_true_SOD_G,donor_date,img_torso,true_SOD_G_torso,img_limbs,true_SOD_G_limbs,index,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,month,season_of_recovery,fall,spring,summer,sex_male
37,004,00400124.12.JPG,2018-01-24,2018-01-09,15.0,67.0,171.0,64.0,3.0,head,00400124,00400124.46.JPG,3.0,00400124.43.JPG,4.0,37,0.000000,0.929097,0.000000,1.026175,2.305528,0.768867,3.735098,1.780798,0.569831,1.908208,0.878973,1.003674,0.000000,1.719278,0.535564,1.127494,2.043196,0.439008,1.267659,1.996106,0.000000,0.083097,1.460189,1.096491,1.924771,0.029327,3.076185,1.413234,0.172621,0.154586,0.991169,1.694158,1.841371,0.795001,0.0,0.666581,0.210517,2.771830,1.368024,0.0,1.148460,0.030999,0.0,1.869062,0.0,0.110060,2.778683,1.543215,0.937168,0.0,0.354045,3.671946,2.741412,1.421189,2.779745,0.356006,0.0,0.000000,1.407682,2.156150,0.162555,1.387765,1.162184,0.013849,01,winter,0,0,0,0
235,004,00400124.38.JPG,2018-01-24,2018-01-09,15.0,67.0,171.0,64.0,3.0,head,00400124,00400124.46.JPG,3.0,00400124.37.JPG,3.0,235,1.301861,0.929097,0.789077,1.128375,1.125460,0.578319,2.321032,2.007199,0.569831,1.225253,0.000000,1.226684,0.121516,0.267010,0.535564,0.660378,1.547907,0.439008,0.332743,1.625679,0.412549,0.000000,0.621157,1.561879,0.470387,0.159045,1.598031,0.742358,0.000000,0.490540,0.808336,0.922500,0.415390,1.559645,0.0,0.596459,1.637382,2.216430,0.829185,0.0,1.547662,0.558798,0.0,0.143504,0.0,0.003905,1.965936,0.310991,0.937168,0.0,0.843907,2.309705,2.070974,1.355797,2.220292,0.423971,0.0,0.393228,1.407682,1.415119,0.648097,0.559703,1.162184,0.013849,01,winter,0,0,0,0
72,004,00400124.13.JPG,2018-01-24,2018-01-09,15.0,67.0,171.0,64.0,3.0,head,00400124,00400124.23.JPG,3.0,00400124.18.JPG,4.0,72,0.000000,1.027153,0.000000,1.008150,2.980005,0.702003,4.128897,1.689883,0.515497,2.960731,0.815381,1.402423,0.000000,2.786601,0.340956,1.564740,1.843078,0.628638,1.358243,2.165700,0.000000,0.259414,2.465603,0.961373,2.476347,0.059002,4.107632,2.085147,0.206266,0.169940,1.175978,1.639801,2.532620,1.018915,0.0,0.558670,0.058538,3.291665,1.496491,0.0,0.984435,0.366301,0.0,3.380263,0.0,0.530783,3.014904,2.328625,0.846786,0.0,0.372075,3.428729,2.927476,1.602061,2.880020,0.296516,0.0,0.000000,1.480884,2.983395,1.177303,1.081967,1.432985,0.000000,01,winter,0,0,0,0
255,004,00400124.39.JPG,2018-01-24,2018-01-09,15.0,67.0,171.0,64.0,3.0,head,00400124,00400124.07.JPG,3.0,00400124.37.JPG,3.0,255,1.301861,1.149898,0.789077,1.341804,1.490941,0.569601,3.135358,2.105528,0.854456,1.169984,0.323624,1.081722,0.121516,0.668364,0.745903,0.660378,1.760769,0.959247,0.553257,1.888132,0.412549,0.000000,0.770173,1.668692,0.630698,0.088827,2.080755,0.742358,0.000000,0.689796,0.753480,1.222801,0.415390,2.376742,0.0,0.560717,1.564825,2.503700,1.075338,0.0,1.735783,0.558798,0.0,0.143504,0.0,0.400011,2.299274,0.310991,0.788177,0.0,0.771736,2.554599,2.474625,1.690201,2.473617,0.512004,0.0,0.393228,1.343125,1.664485,0.719033,1.036993,1.340923,0.000000,01,winter,0,0,0,0
203,004,00400124.38.JPG,2018-01-24,2018-01-09,15.0,67.0,171.0,64.0,3.0,head,00400124,00400124.07.JPG,3.0,00400124.19.JPG,4.0,203,0.000000,1.149898,0.000000,1.239605,2.046739,0.380628,2.978021,1.899740,0.854456,2.883476,0.000000,1.185078,0.000000,2.912200,0.745903,1.371333,1.112744,0.959247,0.332743,1.688022,0.000000,0.410184,2.610728,1.050051,1.643946,0.104207,4.022526,1.892848,0.153347,0.959330,1.375643,0.922500,2.450993,1.612098,0.0,0.560717,0.621101,2.306016,0.444294,0.0,1.832889,0.409887,0.0,3.286015,0.0,0.548375,2.912685,2.268347,0.788177,0.0,0.843907,2.162499,2.231365,1.838398,1.798468,0.423971,0.0,0.000000,1.343125,2.104105,1.137876,0.908860,1.340923,0.000000,01,winter,0,0,0,0


### Drop column not needed

In [75]:
data.columns

Index(['new_id', 'img_head', 'correct_img_date', 'date_placed_ARF', 'PMI_days',
       'age_at_death', 'cadaver_weight_lbs', 'est_stature_in',
       'true_SOD_G_head', 'BP_of_true_SOD_G', 'donor_date', 'img_torso',
       'true_SOD_G_torso', 'img_limbs', 'true_SOD_G_limbs', 'index', '0', '1',
       '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14',
       '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26',
       '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38',
       '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50',
       '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62',
       '63', 'month', 'season_of_recovery', 'fall', 'spring', 'summer',
       'sex_male'],
      dtype='object')

In [76]:
# drop some columns 
data2 = data.drop(['new_id', 'img_head', 'img_torso', 'img_limbs', 'index', 'correct_img_date', 
                   'date_placed_ARF', 'season_of_recovery', 'donor_date', 
           'BP_of_true_SOD_G', 'month', 'season_of_recovery', 'est_stature_in'], axis=1).copy()
data2.head()

,PMI_days,age_at_death,cadaver_weight_lbs,true_SOD_G_head,true_SOD_G_torso,true_SOD_G_limbs,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,fall,spring,summer,sex_male
37,15.0,67.0,171.0,3.0,3.0,4.0,0.000000,0.929097,0.000000,1.026175,2.305528,0.768867,3.735098,1.780798,0.569831,1.908208,0.878973,1.003674,0.000000,1.719278,0.535564,1.127494,2.043196,0.439008,1.267659,1.996106,0.000000,0.083097,1.460189,1.096491,1.924771,0.029327,3.076185,1.413234,0.172621,0.154586,0.991169,1.694158,1.841371,0.795001,0.0,0.666581,0.210517,2.771830,1.368024,0.0,1.148460,0.030999,0.0,1.869062,0.0,0.110060,2.778683,1.543215,0.937168,0.0,0.354045,3.671946,2.741412,1.421189,2.779745,0.356006,0.0,0.000000,1.407682,2.156150,0.162555,1.387765,1.162184,0.013849,0,0,0,0
235,15.0,67.0,171.0,3.0,3.0,3.0,1.301861,0.929097,0.789077,1.128375,1.125460,0.578319,2.321032,2.007199,0.569831,1.225253,0.000000,1.226684,0.121516,0.267010,0.535564,0.660378,1.547907,0.439008,0.332743,1.625679,0.412549,0.000000,0.621157,1.561879,0.470387,0.159045,1.598031,0.742358,0.000000,0.490540,0.808336,0.922500,0.415390,1.559645,0.0,0.596459,1.637382,2.216430,0.829185,0.0,1.547662,0.558798,0.0,0.143504,0.0,0.003905,1.965936,0.310991,0.937168,0.0,0.843907,2.309705,2.070974,1.355797,2.220292,0.423971,0.0,0.393228,1.407682,1.415119,0.648097,0.559703,1.162184,0.013849,0,0,0,0
72,15.0,67.0,171.0,3.0,3.0,4.0,0.000000,1.027153,0.000000,1.008150,2.980005,0.702003,4.128897,1.689883,0.515497,2.960731,0.815381,1.402423,0.000000,2.786601,0.340956,1.564740,1.843078,0.628638,1.358243,2.165700,0.000000,0.259414,2.465603,0.961373,2.476347,0.059002,4.107632,2.085147,0.206266,0.169940,1.175978,1.639801,2.532620,1.018915,0.0,0.558670,0.058538,3.291665,1.496491,0.0,0.984435,0.366301,0.0,3.380263,0.0,0.530783,3.014904,2.328625,0.846786,0.0,0.372075,3.428729,2.927476,1.602061,2.880020,0.296516,0.0,0.000000,1.480884,2.983395,1.177303,1.081967,1.432985,0.000000,0,0,0,0
255,15.0,67.0,171.0,3.0,3.0,3.0,1.301861,1.149898,0.789077,1.341804,1.490941,0.569601,3.135358,2.105528,0.854456,1.169984,0.323624,1.081722,0.121516,0.668364,0.745903,0.660378,1.760769,0.959247,0.553257,1.888132,0.412549,0.000000,0.770173,1.668692,0.630698,0.088827,2.080755,0.742358,0.000000,0.689796,0.753480,1.222801,0.415390,2.376742,0.0,0.560717,1.564825,2.503700,1.075338,0.0,1.735783,0.558798,0.0,0.143504,0.0,0.400011,2.299274,0.310991,0.788177,0.0,0.771736,2.554599,2.474625,1.690201,2.473617,0.512004,0.0,0.393228,1.343125,1.664485,0.719033,1.036993,1.340923,0.000000,0,0,0,0
203,15.0,67.0,171.0,3.0,3.0,4.0,0.000000,1.149898,0.000000,1.239605,2.046739,0.380628,2.978021,1.899740,0.854456,2.883476,0.000000,1.185078,0.000000,2.912200,0.745903,1.371333,1.112744,0.959247,0.332743,1.688022,0.000000,0.410184,2.610728,1.050051,1.643946,0.104207,4.022526,1.892848,0.153347,0.959330,1.375643,0.922500,2.450993,1.612098,0.0,0.560717,0.621101,2.306016,0.444294,0.0,1.832889,0.409887,0.0,3.286015,0.0,0.548375,2.912685,2.268347,0.788177,0.0,0.843907,2.162499,2.231365,1.838398,1.798468,0.423971,0.0,0.000000,1.343125,2.104105,1.137876,0.908860,1.340923,0.000000,0,0,0,0


### Deal with missing values

In [77]:
# drop rows with missing values
data2.dropna(inplace=True)
data2.shape

(984, 74)

In [78]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 984 entries, 37 to 41978
Data columns (total 74 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   PMI_days            984 non-null    float64
 1   age_at_death        984 non-null    float64
 2   cadaver_weight_lbs  984 non-null    float64
 3   true_SOD_G_head     984 non-null    float64
 4   true_SOD_G_torso    984 non-null    float64
 5   true_SOD_G_limbs    984 non-null    float64
 6   0                   984 non-null    float64
 7   1                   984 non-null    float64
 8   2                   984 non-null    float64
 9   3                   984 non-null    float64
 10  4                   984 non-null    float64
 11  5                   984 non-null    float64
 12  6                   984 non-null    float64
 13  7                   984 non-null    float64
 14  8                   984 non-null    float64
 15  9                   984 non-null    float64
 16  10   

In [79]:
data_processed = data2.copy()

# Final dataset construction
Models will be built for varying number of features, including:
1. Demographics only 
2. Demographics + weather aggregates
3. Demographics + weather aggregates + img embeddings

In all of the above cases, SOD features (true_SOD_G_head, true_SOD_G_torso, and true_SOD_G_limbs) can be included or not.

### Demographics 

In [87]:
# create X and y dataframes
X = data_processed.drop('PMI_days', axis=1)
display(X.head(2))
print(X.shape)

y = data_processed[['PMI_days']]
display(y.head(2))
print(y.shape)

,age_at_death,cadaver_weight_lbs,true_SOD_G_head,true_SOD_G_torso,true_SOD_G_limbs,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,fall,spring,summer,sex_male
37,67.0,171.0,3.0,3.0,4.0,0.000000,0.929097,0.000000,1.026175,2.305528,0.768867,3.735098,1.780798,0.569831,1.908208,0.878973,1.003674,0.000000,1.719278,0.535564,1.127494,2.043196,0.439008,1.267659,1.996106,0.000000,0.083097,1.460189,1.096491,1.924771,0.029327,3.076185,1.413234,0.172621,0.154586,0.991169,1.694158,1.841371,0.795001,0.0,0.666581,0.210517,2.77183,1.368024,0.0,1.148460,0.030999,0.0,1.869062,0.0,0.110060,2.778683,1.543215,0.937168,0.0,0.354045,3.671946,2.741412,1.421189,2.779745,0.356006,0.0,0.000000,1.407682,2.156150,0.162555,1.387765,1.162184,0.013849,0,0,0,0
235,67.0,171.0,3.0,3.0,3.0,1.301861,0.929097,0.789077,1.128375,1.125460,0.578319,2.321032,2.007199,0.569831,1.225253,0.000000,1.226684,0.121516,0.267010,0.535564,0.660378,1.547907,0.439008,0.332743,1.625679,0.412549,0.000000,0.621157,1.561879,0.470387,0.159045,1.598031,0.742358,0.000000,0.490540,0.808336,0.922500,0.415390,1.559645,0.0,0.596459,1.637382,2.21643,0.829185,0.0,1.547662,0.558798,0.0,0.143504,0.0,0.003905,1.965936,0.310991,0.937168,0.0,0.843907,2.309705,2.070974,1.355797,2.220292,0.423971,0.0,0.393228,1.407682,1.415119,0.648097,0.559703,1.162184,0.013849,0,0,0,0


(984, 73)


,PMI_days
37,15.0
235,15.0


(984, 1)


In [85]:
# split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                   random_state=1,  
                                   test_2size=0.2,  
                                   shuffle=True)
print(len(X_train), len(y_train), len(X_test), len(y_test))

787 787 197 197


In [86]:
'''
X_train.to_pickle('../data/Gelderman_SOD_cohort/demographics/X_train.pkl')
y_train.to_pickle('../data/Gelderman_SOD_cohort/demographics/y_train.pkl')
X_test.to_pickle('../data/Gelderman_SOD_cohort/demographics/X_test.pkl')
y_test.to_pickle('../data/Gelderman_SOD_cohort/demographics/y_test.pkl')
'''

### Demographics + weather aggregates

In [ ]:
# create X and y dataframes
X = data_processed.drop('PMI_days', axis=1)
display(X.head(2))
print(X.shape)

y = data_processed[['PMI_days']]
display(y.head(2))
print(y.shape)

In [ ]:
'''
X_train.to_pickle('../data/Gelderman_SOD_cohort/demographics_weather/X_train.pkl')
y_train.to_pickle('../data/Gelderman_SOD_cohort/demographics_weather/y_train.pkl')
X_test.to_pickle('../data/Gelderman_SOD_cohort/demographics_weather/X_test.pkl')
y_test.to_pickle('../data/Gelderman_SOD_cohort/demographics_weather/y_test.pkl')
'''

### Demographics + weather aggregates + image embeddings

In [ ]:
# create X and y dataframes
X = data_processed.drop('PMI_days', axis=1)
display(X.head(2))
print(X.shape)

y = data_processed[['PMI_days']]
display(y.head(2))
print(y.shape)

In [ ]:
'''
X_train.to_pickle('../data/Gelderman_SOD_cohort/demographics_weather_imgs/X_train.pkl')
y_train.to_pickle('../data/Gelderman_SOD_cohort/demographics_weather_imgs/y_train.pkl')
X_test.to_pickle('../data/Gelderman_SOD_cohort/demographics_weather_imgs/X_test.pkl')
y_test.to_pickle('../data/Gelderman_SOD_cohort/demographics_weather_imgs/y_test.pkl')
'''